In [1]:
%pip install pandas scikit-learn
%pip install numpy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pickle
import numpy as np

In [3]:
nba_data = pd.read_csv('data/nba_basic_data.csv')

X = nba_data.drop(['PTS'], axis=1)
y = nba_data['PTS']

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
X_train

,PlayerID,TeamID,OpponentID,Location,Playoffs,WinOrLoss,Starter,PlayerType,PerfScore,MIN,...,FTA,REB,AST,STL,BLK,OREB,TO,PF,opp_def_rtg,opp_def_rtg_adj
23816,3441,6,22,0,0,0,0,0,9,25.0,...,2,3,4,2,0,1,1,1,107.35,107.77
13600,1754,6,5,0,0,0,1,0,6,34.0,...,0,7,0,0,1,0,0,0,104.40,104.51
20806,4348,2,35,1,0,0,1,0,9,42.0,...,6,8,3,2,0,0,0,0,106.18,105.89
67675,3893,2,36,1,0,1,0,0,19,38.0,...,8,14,3,3,3,1,3,5,110.36,110.56
27936,1559,6,24,1,0,1,0,0,1,16.0,...,0,2,0,0,0,1,0,4,109.47,109.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17738,2037,17,16,0,1,1,0,0,0,2.0,...,0,0,0,0,0,0,0,0,106.75,107.42
39220,119815,15,17,1,0,0,0,0,3,18.0,...,0,2,1,0,0,0,1,0,104.25,104.12
164883,1825,23,22,1,0,1,1,0,9,29.0,...,0,3,5,1,4,1,0,1,119.51,119.48
99682,3572,5,16,1,0,1,1,0,5,30.0,...,4,4,2,1,0,0,3,1,111.98,112.06


In [84]:
from sklearn.ensemble import RandomForestRegressor

In [85]:
rfr = RandomForestRegressor(random_state=13, n_jobs=-1, verbose=True)

rfr.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.5s finished


RandomForestRegressor(n_jobs=-1, random_state=13, verbose=True)

In [86]:
y_pred = rfr.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [87]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [88]:
float(mean_absolute_error(y_pred, y_test))

0.01359674287866934

In [89]:
float(mean_squared_error(y_pred, y_test))

0.021107632791445712

In [90]:
float(r2_score(y_pred, y_test))

0.9997467623864458

In [91]:
params = {
    'n_estimators' : [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

In [92]:
from sklearn.model_selection import GridSearchCV

In [95]:
rfr_cv = GridSearchCV(estimator=rfr, cv=3, param_grid=params, scoring="neg_mean_squared_error", n_jobs=-1, verbose=True)

In [ ]:
rfr_cv.fit(X_train, y_train)

In [114]:
y_cv_pred = rfr_cv.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.1s finished


array([ 8.        ,  5.        , 19.        , ..., 16.97666667,
        2.        , 17.        ])

In [106]:
float(mean_absolute_error(y_cv_pred, y_test))

0.01312623759985318

In [107]:
float(mean_squared_error(y_cv_pred, y_test))

0.019399927233153348

In [108]:
float(r2_score(y_cv_pred, y_test))

0.9997672569036895

In [113]:
with open("models/nba_random_forest_one_target_basic_data.pkl", 'wb') as file:
    pickle.dump(rfr_cv, file)

In [11]:
with open("models/nba_random_forest_one_target_basic_data.pkl", "rb") as file:
    rfr_cv = pickle.load(file)

In [15]:
test_data = pd.DataFrame({'PlayerID' : 1871,
             'TeamID': 16,
             'OpponentID': 5,
             'Location': 1,
             'Playoffs': 0,
             'WinOrLoss': 1,
             'Starter': 0,
             'PlayerType': 0,
             'PerfScore': 20,
             'MIN': 31.0,
             'FGM': 5,
             'FGA': 33,
             '3FM': 1,
             '3FA': 20,
             'FTM': 1, 
             'FTA': 16,
             'REB': 10, 
             'AST': 7, 
             'STL': 1,
             'BLK': 4, 
             'OREB': 0, 
             'TO': 0, 
             'PF': 5,
             'opp_def_rtg': 120.25,
             'opp_def_rtg_adj': 120.45}, index=[0])

# 647935,23,6,1,0,1,1,0,20,44.0,73,25,33,8,13,15,16,10,7,1,4,0,0,1,120.25,120.45

rfr_cv.predict(test_data)


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.0s finished


array([12.])